In [16]:
import os
import logging
from pathlib import Path

import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
import sys
from joblib import Parallel, delayed
logging.basicConfig(level=logging.ERROR)


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [17]:
# 
class CFG:
    def __init__(self, mode="train", kaggle_notebook=False, debug=False):
        assert mode in ["train", "inference"], "mode must be 'train' or 'inference'"
        self.mode = mode
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
            self.model_path = "/kaggle/input/birdclef-2025-baseline-fold0-0404"
            
            self.device = "cpu"
            self.batch_size = 8
            self.n_jobs = 3
            
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = '../data/raw/test_soundscapes/'
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.spectrogram_npy = '../data/processed/mel-spec_0329/birdclef2025_melspec_5sec_256_256.npy'
            self.MODELS_DIR = "../models/"
            self.model_path =  "../models/mel_cleaned_contrast02_vino/"
            
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.batch_size = 32
            self.n_jobs = 3

        # ===== Model Settings =====
        self.model_name = 'efficientnet_b0'
        self.pretrained = True if mode == "train" else False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.FS = 32000
        self.WINDOW_SIZE = 5
        self.TARGET_DURATION = 5
        self.TARGET_SHAPE = (256, 256)
        self.N_FFT = 1024
        self.HOP_LENGTH = 64
        self.N_MELS = 148
        self.FMIN = 20
        self.FMAX = 16000
        
        self.seed = 42
        
        self.smooth_center_weight = 0.6
        self.smooth_neighbor_weight = 0.2
        
        # ===== Inference Mode =====
        if mode == "inference":
            self.use_tta = False
            self.tta_count = 3
            self.threshold = 0.5

            self.use_specific_folds = True
            self.folds = [0, 3, 4]  # Used only if use_specific_folds is True

            self.debug_count = 3
            
            self.is_contrast = True
            self.contrast_factor = 0.2
            
            
    def update_debug_settings(self):
        if self.debug:
            self.epochs = 2
            self.selected_folds = [0]

In [18]:
"TODO: configを2つにわけるべきかも．柔軟に変える方はnotebook側で，固定したい方はmodule側とか"
"TODO: Debugモードになっていたらsubmissionでエラーになる"
"TODO: 疑似ラベル作成のためなら，事前にデータセットを準備するべき"
cfg = CFG(mode='inference', kaggle_notebook=False)

if cfg.KAGGLE_NOTEBOOK:
    !pip install -U openvino-telemetry  --no-index --find-links /kaggle/input/pip-hub
    !pip install -U openvino  --no-index --find-links /kaggle/input/pip-hub
    sys.path.append("/kaggle/input/birdclef-2025-libs/")
    
from openvino.runtime import Core
from module import models_lib, utils_lib, preprocess_lib, inference_lib

# Set seed
utils_lib.set_seed(cfg.seed)

In [19]:
print(f"Using device: {cfg.device}")
print(f"Loading taxonomy data...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)
print(f"Number of classes: {num_classes}")

Using device: cuda
Loading taxonomy data...
Number of classes: 206


In [20]:
def process_audio_file(audio_path, cfg):
    """1ファイル分のmelspecデータを返す（row_id, melspecのリスト）"""
    dataset = []
    soundscape_id = Path(audio_path).stem
    try:
        audio_data, _ = librosa.load(audio_path, sr=cfg.FS)
        total_segments = int(len(audio_data) / (cfg.FS * cfg.WINDOW_SIZE))

        for segment_idx in range(total_segments):
            start = int(segment_idx * cfg.FS * cfg.WINDOW_SIZE)
            end = int(start + cfg.FS * cfg.WINDOW_SIZE)
            segment_audio = audio_data[start:end]

            mel_spec = preprocess_lib.process_audio_segment(segment_audio, cfg)
            row_id = f"{soundscape_id}_{(segment_idx + 1) * cfg.WINDOW_SIZE}"

            dataset.append({
                "row_id": row_id,
                "mel_spec": mel_spec
            })
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
    return dataset

def generate_melspec_dataset(cfg):
    test_dir = Path(cfg.test_soundscapes)
    if not test_dir.exists():
        print(f"Test directory {test_dir} does not exist.")
        return []

    test_files = list(test_dir.glob('*.ogg'))
    if len(test_files) == 0:
        print("No test audio files found.")
        return []

    if cfg.debug:
        print(f"Debug mode enabled, using only {cfg.debug_count} files")
        test_files = test_files[:cfg.debug_count]

    results = Parallel(n_jobs=cfg.n_jobs)(
        delayed(process_audio_file)(path, cfg) for path in tqdm(test_files, desc="Parallel melspec gen")
    )
    dataset = [item for sublist in results for item in sublist]
    return dataset

def apply_contrast_to_dataset(dataset, factor=0.5):
    """
    dataset（list of dicts）の 'mel_spec' にコントラストを適用し、新しいリストを返す。
    dataset が空でも安全に処理。

    :param dataset: [{"row_id": ..., "mel_spec": ...}, ...]
    :param factor: コントラスト強度（例: 0.5）
    :return: 新しい dataset（コントラスト強調済み）または空リスト
    """
    if not dataset:
        print("⚠️ apply_contrast_to_dataset: dataset is empty. Returning empty list.")
        return []
    print(f"Applying contrast with factor {factor} to dataset of length {len(dataset)}")
    def enhance(spec, factor=0.5):
        mean = np.mean(spec)
        enhanced = mean + (spec - mean) * (1 + factor)
        return np.clip(enhanced, 0, 1)

    new_dataset = []
    for item in dataset:
        new_dataset.append({
            "row_id": item["row_id"],
            "mel_spec": enhance(item["mel_spec"], factor)
        })

    return new_dataset

In [21]:
def load_openvino_models(vino_dir, cfg):
    models = []
    vino_dir = Path(vino_dir)

    if cfg.use_specific_folds:
        fold_ids = cfg.folds
        xml_files = [vino_dir / f"model_fold{f}.xml" for f in fold_ids]
    else:
        xml_files = sorted(vino_dir.glob("model_fold*.xml"))

    for xml_path in xml_files:
        bin_path = xml_path.with_suffix(".bin")

        if not xml_path.exists() or not bin_path.exists():
            print(f"⚠️ Warning: Missing files for {xml_path.stem}")
            continue

        core = Core()
        model_ir = core.read_model(xml_path)
        compiled_model = core.compile_model(model_ir, device_name="CPU")
        models.append(compiled_model)

        # 🔍 モデルのファイル名（fold情報）をログに出す
        print(f"✅ Loaded model: {xml_path.name}")

    print(f"🎉 Total {len(models)} OpenVINO model(s) loaded from {vino_dir}")
    return models

# === NEW ===
def run_inference_openvino(dataset, models_ir, cfg, species_ids):
    row_ids = []
    all_preds = []

    for i in range(0, len(dataset), cfg.batch_size):
        batch = dataset[i:i+cfg.batch_size]

        mel_list = [item["mel_spec"] for item in batch]
        input_tensor = np.stack(mel_list).astype(np.float32)  # (B, H, W)
        input_tensor = np.expand_dims(input_tensor, axis=1)  # (B, 1, H, W)

        preds_per_model = []
        for model in models_ir:
            input_layer = model.input(0)
            output_layer = model.output(0)
            result = model([input_tensor])[output_layer]
            probs = 1 / (1 + np.exp(-result))  # sigmoid
            preds_per_model.append(probs)

        avg_preds = np.mean(preds_per_model, axis=0)
        all_preds.append(avg_preds)
        row_ids.extend([item["row_id"] for item in batch])

    predictions = np.concatenate(all_preds, axis=0)
    return row_ids, predictions

In [22]:
def smooth_submission_df(submission_df, cfg):
    """
    Smooth predictions in a submission DataFrame and return the modified DataFrame.

    Each row's prediction is averaged with its neighbors using weights from cfg.

    :param submission_df: DataFrame containing submission data (row_id + prediction columns)
    :param cfg: Configuration object containing smoothing weights
    :return: Smoothed submission DataFrame
    """
    print("Smoothing submission predictions...")
    sub = submission_df.copy()
    cols = sub.columns[1:]
    groups = sub['row_id'].astype(str).str.rsplit('_', n=1).str[0].values
    unique_groups = np.unique(groups)

    for group in unique_groups:
        idx = np.where(groups == group)[0]
        sub_group = sub.iloc[idx].copy()
        predictions = sub_group[cols].values
        new_predictions = predictions.copy()

        if predictions.shape[0] > 1:
            w_c = cfg.smooth_center_weight
            w_n = cfg.smooth_neighbor_weight
            new_predictions[0] = predictions[0] * (1 - w_n) + predictions[1] * w_n
            new_predictions[-1] = predictions[-1] * (1 - w_n) + predictions[-2] * w_n
            for i in range(1, predictions.shape[0] - 1):
                new_predictions[i] = (
                    predictions[i - 1] * w_n +
                    predictions[i] * w_c +
                    predictions[i + 1] * w_n
                )
        sub.iloc[idx, 1:] = new_predictions

    return sub

In [23]:
print("Generating dataset...")
dataset = generate_melspec_dataset(cfg)

if cfg.is_contrast:
    dataset = apply_contrast_to_dataset(dataset, factor=cfg.contrast_factor)
else:
    print("No contrast applied to dataset.")

print("Loading OpenVINO models...")
vino_dir = Path(cfg.model_path).with_name(Path(cfg.model_path).name)
models_ir = load_openvino_models(vino_dir, cfg)

if not models_ir:
    raise RuntimeError("No OpenVINO models found.")

print("Running OpenVINO inference...")
if len(dataset) > 0:
    row_ids, predictions = run_inference_openvino(dataset, models_ir, cfg, species_ids)
else:
    print("No test data available, generating empty submission.")
    row_ids = []
    predictions = []

submission_df = utils_lib.create_submission(row_ids, predictions, species_ids, cfg)
submission_path = os.path.join(cfg.OUTPUT_DIR, 'submission_before_smoothing.csv')
submission_df.to_csv(submission_path, index=False)

print(f"Submission saved to {submission_path}")


Generating dataset...


Parallel melspec gen:   0%|          | 0/71 [00:00<?, ?it/s]

Applying contrast with factor 0.2 to dataset of length 852
Loading OpenVINO models...
✅ Loaded model: model_fold0.xml
✅ Loaded model: model_fold3.xml
✅ Loaded model: model_fold4.xml
🎉 Total 3 OpenVINO model(s) loaded from ../models/mel_cleaned_contrast02_vino
Running OpenVINO inference...
Creating submission dataframe...
Submission saved to ../data/result/submission_before_smoothing.csv


In [24]:
# Smooth and save separately
smoothed_df = smooth_submission_df(submission_df, cfg)
smoothed_submission_path = os.path.join(cfg.OUTPUT_DIR, 'submission.csv')
smoothed_df.to_csv(smoothed_submission_path, index=False)
print(f"Smoothed submission saved to {smoothed_submission_path}")

Smoothing submission predictions...
Smoothed submission saved to ../data/result/submission.csv


In [25]:
# 提出用ファイルを読み込む
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission.csv'))
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.000934,0.000368,0.000469,0.000335,0.000858,0.003740,0.001472,0.000804,0.003028,...,0.008125,0.002110,0.001045,0.002041,0.000120,0.003281,0.000959,0.010329,0.002770,0.003845
1,H02_20230420_074000_10,0.001166,0.000497,0.000594,0.000412,0.001241,0.003457,0.001770,0.000979,0.004027,...,0.007502,0.001747,0.000994,0.001248,0.000188,0.004428,0.001409,0.007870,0.001397,0.002748
2,H02_20230420_074000_15,0.001660,0.000759,0.000847,0.000713,0.001672,0.003154,0.003070,0.001355,0.008991,...,0.011368,0.002740,0.001335,0.000909,0.000377,0.006202,0.002129,0.006596,0.001096,0.001519
3,H02_20230420_074000_20,0.001960,0.001171,0.001130,0.000481,0.002020,0.003588,0.003122,0.001674,0.014716,...,0.010888,0.004103,0.000956,0.001228,0.000250,0.005558,0.001898,0.008149,0.000937,0.001437
4,H02_20230420_074000_25,0.001628,0.001021,0.000913,0.000463,0.002023,0.003731,0.003201,0.001437,0.007585,...,0.008734,0.002455,0.000735,0.001463,0.000113,0.005952,0.001386,0.011615,0.001253,0.003089


In [26]:
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission_before_smoothing.csv'))
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.000899,0.000336,0.000442,0.000361,0.000763,0.003790,0.001556,0.000775,0.002941,...,0.008923,0.002291,0.001134,0.002286,0.000127,0.003160,0.000891,0.011093,0.003260,0.004114
1,H02_20230420_074000_10,0.001073,0.000499,0.000580,0.000232,0.001236,0.003540,0.001134,0.000920,0.003374,...,0.004935,0.001385,0.000691,0.001064,0.000096,0.003766,0.001231,0.007273,0.000810,0.002769
2,H02_20230420_074000_15,0.001711,0.000651,0.000786,0.001003,0.001730,0.002876,0.003890,0.001357,0.007070,...,0.013782,0.002288,0.001763,0.000762,0.000527,0.007680,0.002459,0.006439,0.001294,0.001318
3,H02_20230420_074000_20,0.002096,0.001345,0.001296,0.000323,0.001934,0.003604,0.002546,0.001786,0.020369,...,0.010561,0.005451,0.000694,0.001197,0.000207,0.004201,0.002037,0.006388,0.000788,0.000874
4,H02_20230420_074000_25,0.001803,0.001168,0.000975,0.000435,0.002565,0.004251,0.004081,0.001656,0.005405,...,0.008974,0.001876,0.000932,0.001785,0.000102,0.007505,0.000922,0.015142,0.001025,0.003244


In [27]:
submission

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.000899,0.000336,0.000442,0.000361,0.000763,0.003790,0.001556,0.000775,0.002941,...,0.008923,0.002291,0.001134,0.002286,0.000127,0.003160,0.000891,0.011093,0.003260,0.004114
1,H02_20230420_074000_10,0.001073,0.000499,0.000580,0.000232,0.001236,0.003540,0.001134,0.000920,0.003374,...,0.004935,0.001385,0.000691,0.001064,0.000096,0.003766,0.001231,0.007273,0.000810,0.002769
2,H02_20230420_074000_15,0.001711,0.000651,0.000786,0.001003,0.001730,0.002876,0.003890,0.001357,0.007070,...,0.013782,0.002288,0.001763,0.000762,0.000527,0.007680,0.002459,0.006439,0.001294,0.001318
3,H02_20230420_074000_20,0.002096,0.001345,0.001296,0.000323,0.001934,0.003604,0.002546,0.001786,0.020369,...,0.010561,0.005451,0.000694,0.001197,0.000207,0.004201,0.002037,0.006388,0.000788,0.000874
4,H02_20230420_074000_25,0.001803,0.001168,0.000975,0.000435,0.002565,0.004251,0.004081,0.001656,0.005405,...,0.008974,0.001876,0.000932,0.001785,0.000102,0.007505,0.000922,0.015142,0.001025,0.003244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,H02_20230502_080500_40,0.000045,0.000053,0.000038,0.000035,0.000077,0.000019,0.000042,0.000023,0.000011,...,0.003151,0.007229,0.000169,0.003003,0.000332,0.000407,0.003940,0.000915,0.000281,0.000178
848,H02_20230502_080500_45,0.000140,0.000078,0.000127,0.000045,0.000144,0.000183,0.000415,0.000107,0.000053,...,0.009498,0.002814,0.004892,0.004091,0.001527,0.009769,0.004200,0.002720,0.010019,0.002426
849,H02_20230502_080500_50,0.000063,0.000071,0.000035,0.000004,0.000029,0.000054,0.000247,0.000044,0.000023,...,0.032925,0.003485,0.002133,0.004819,0.000154,0.003951,0.001224,0.008669,0.008807,0.000638
850,H02_20230502_080500_55,0.000048,0.000073,0.000031,0.000053,0.000077,0.000038,0.000045,0.000027,0.000012,...,0.000773,0.001655,0.000053,0.006788,0.000280,0.000173,0.007872,0.000282,0.000431,0.000463


In [28]:
print("✅ Shape:", submission.shape)
print("✅ Columns:", submission.columns.tolist())
print("✅ Dtypes:\n", submission.dtypes)
print("✅ Nulls:\n", submission.isna().sum().sum())

✅ Shape: (852, 207)
✅ Columns: ['row_id', '1139490', '1192948', '1194042', '126247', '1346504', '134933', '135045', '1462711', '1462737', '1564122', '21038', '21116', '21211', '22333', '22973', '22976', '24272', '24292', '24322', '41663', '41778', '41970', '42007', '42087', '42113', '46010', '47067', '476537', '476538', '48124', '50186', '517119', '523060', '528041', '52884', '548639', '555086', '555142', '566513', '64862', '65336', '65344', '65349', '65373', '65419', '65448', '65547', '65962', '66016', '66531', '66578', '66893', '67082', '67252', '714022', '715170', '787625', '81930', '868458', '963335', 'amakin1', 'amekes', 'ampkin1', 'anhing', 'babwar', 'bafibi1', 'banana', 'baymac', 'bbwduc', 'bicwre1', 'bkcdon', 'bkmtou1', 'blbgra1', 'blbwre1', 'blcant4', 'blchaw1', 'blcjay1', 'blctit1', 'blhpar1', 'blkvul', 'bobfly1', 'bobher1', 'brtpar1', 'bubcur1', 'bubwre1', 'bucmot3', 'bugtan', 'butsal1', 'cargra1', 'cattyr', 'chbant1', 'chfmac1', 'cinbec1', 'cocher1', 'cocwoo1', 'colara1', '